Game Play
To play a hand of Blackjack the following steps must be followed:

Create a deck of 52 cards

Shuffle the deck

Ask the Player for their bet

Make sure that the Player's bet does not exceed their available chips

Deal two cards to the Dealer and two cards to the Player

Show only one of the Dealer's cards, the other remains hidden

Show both of the Player's cards

Ask the Player if they wish to Hit, and take another card

If the Player's hand doesn't Bust (go over 21), ask if they'd like to Hit again.

If a Player Stands, play the Dealer's hand. The dealer will always Hit until the Dealer's value meets or exceeds 17

Determine the winner and adjust the Player's chips accordingly

Ask the Player if they'd like to play again



In [3]:
import itertools
import random
from itertools import combinations,product


In [79]:
class Card(object):
    rank_to_value =  {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

    def __init__(self, suit, rank):
        self._suit = suit
        self.rank = rank
    
    def __str__(self):
        return f"{self.rank} of {self._suit}"
     
    @property
    def value(self):
        return self.rank_to_value[self.rank]
    
    def to_hidden(self):
        return HiddenCard(self._suit, self.rank)
    
    def to_card(self):
        return self
    
class HiddenCard(Card):
    
    def __str__(self):
        return "<Hidden Card>"
     
    @property
    def value(self):
        return None
    
    def to_card(self):
        return Card(self._suit, self.rank)
    
        
class Deck:
    """
    This class creates a deck and it has two functions.
    The first function is to shuffle the deck and the other one  draws a card from the deck.
    """
    suit = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
    rank = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
    
    def __init__(self):
        self.cards = list(itertools.product(self.suit,self.rank))
        self.deck=[]
        for suit, rank in self.cards:
            self.deck.append(Card(suit, rank))        
          
    def shuffle(self):
        random.shuffle(self.deck)
    
    def draw_card(self, hidden=False):
        card = self.deck.pop(0)
        if hidden:
            card = card.to_hidden() 
        return card
    
    def __str__(self):
        return str(self.deck)

class Hand:
    
    def __init__(self):
        self.cards_on_hand = []
        
    def __str__(self):
        str_of_hand = '' 
        for card in self.cards_on_hand:
            str_of_hand += str(card)+'\n'
        return str(str_of_hand)
                    
    def add_card(self, card):
        self.cards_on_hand.append(card)
        
    def show_hidden_cards(self):
        self.cards_on_hand = [card.to_card() for card in self.cards_on_hand]

    @property
    def points_on_hand(self):
        """
        The input "cards_on_hand" must be a list with cards. This method calculates the total sum of points a player has.
        """
        sum_of_points = 0
        for card in self.cards_on_hand:
            sum_of_points += card.value
            for card in self.cards_on_hand:
                if card.rank == 'Ace':
                    if sum_of_points > 21:
                        sum_of_points -= 10           
        return sum_of_points
    @property
    def is_bust(self):
        if self.points_on_hand > 21:
            return True
        else:
            return False
    @property
    def is_win(self):
        if self.points_on_hand == 21:
            return True
        else:
            return False
class Player:
    
    def __init__(self):
        self.cash = 1000
        self.hand = Hand()

    def __getattr__(self, key):
        return getattr(self.hand, key)
        
    def __str__(self):
        return f"You are playing a game of Black Jack. You currently have {self.cash}$ left."

    def betting(self):
        print('betting with cash ', self.cash)
        bet = int(input(f"You have {self.cash} $. How much do you want to bet? :"))
        while bet > self.cash or bet <= 0:
            print(f"Sorry you can't make that bet. You have {self.cash} $ left.")
            bet=int(input("Place a new bet: "))            
        
        self.cash -= bet
        print(f"You have made a bet on {bet} $ and you have {self.cash} $ left in your wallet.")
        return bet
    
    def winning_profit(self, profit):
        self.cash += profit
    
    def hit_or_stand(self):
        while True:
            action = input('Do you want to hit or stand ? [h/s)] : ')
            if action == 'h' or 's':
                break
        if action =='h':
            return True
        elif action =='s':
            return False


class Dealer:  
    def __init__(self):
        self.hand = Hand()
    
    def __str__(self):
        return "Dealer has cards: " + str(self.hand)
    
    def __getattr__(self, key):
        return getattr(self.hand, key)

    def hit_or_stand(self):
        self.has_hidden_card = False
        if self.hand.points_on_hand >= 17:
            return False
        else:
            return True
        
class GameOn:
    def __init__(self):
        self.player = Player()
        self.dealer = Dealer()
        self.deck = Deck()
    
    def __str__(self):
        return "The game is on!"
    
    def welcome(self):
        print('Hello and welcome to this amazing game of blackjack! Get as close to 21 as you can without going over! \nDealer hits until she reaches 17. Aces count as 1 or 11. You have 1000$ to bet')
    
    def win_check(self):
        """
        Returns a tuple containing a boolean with information about who won the game. 
        If first index is True that means that the player won. 
        If second index is True that means that the dealer won.
        If both index are False that means that it is a tie.
        """
        player_win=False
        dealer_win=False

        if self.player.is_win:
            player_win = True
        elif 21 >= self.player.points_on_hand and self.player.points_on_hand > self.dealer.points_on_hand:
            player_win = True
        elif self.dealer.points_on_hand > 21 and self.player.points_on_hand<22:
            player_win = True

        elif self.dealer.points_on_hand == 21 :
            dealer_win = True
        elif self.player.points_on_hand < self.dealer.points_on_hand and self.dealer.points_on_hand <= 21:
            dealer_win = True
        elif self.player.points_on_hand > 21 and self.player.points_on_hand <= 21:
            dealer_win = True
        return (player_win, dealer_win)
    
    def game_over(self):  
        new_game = input('Do you want to play again? [y/n]: ')
        if new_game == 'y':
            self.player.hand = Hand()
            self.dealer.hand = Hand()
            self.play()
        else:
            print("Ok. Game Over.")
    
    def play(self):
        """
        This function executes the gameplay.
        """
        self.welcome()
        while True:
            profit = self.player.betting()*2
            self.deck.shuffle()
            self.player.add_card(self.deck.draw_card())
            self.player.add_card(self.deck.draw_card())
            self.dealer.add_card(self.deck.draw_card())
            self.dealer.add_card(self.deck.draw_card(hidden=True))
            print('\nPlayers Hand: \n'+ str(self.player.hand))
            print('\nDealers Hand: \n'+ str(self.dealer.hand))
        
            while self.player.hit_or_stand() == True:
                self.player.add_card(self.deck.draw_card())
                print('\nPlayers Hand: \n'+ str(self.player.hand))
                print('\nDealers Hand: \n'+ str(self.dealer.hand))

                if self.player.hand.is_win == True:
                    print("Congratulations! You have won the game!")
                    winners_reward
                    self.game_over()
                elif self.player.hand.is_bust == True:
                    print("You bust!")
                    self.game_over()
                    
            self.dealer.hand.show_hidden_cards()
            self.dealer.add_card(self.deck.draw_card())
            print('\nPlayers Hand: \n'+ str(self.player.hand))
            print('\nDealers Hand: \n'+ str(self.dealer.hand))
            
            if self.win_check() == (True, False):
                print("Congratulations! You have won the game!")
                self.player.winning_profit(profit)
                
            elif self.win_check() == (False, True):
                print("You lose!")
            else:
                print("It's a tie!")

            self.game_over()